In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib

In [3]:
%cd /content/drive/MyDrive/Colab/DATN/data-modeling

/content/drive/MyDrive/Colab/DATN/data-modeling


In [4]:
!pip install flask flask-ngrok

In [5]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import pickle
import os
import pandas as pd

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Load the models and label encoders
save_dir = '/content/drive/MyDrive/Colab/DATN/data-modeling/'

with open('/content/drive/MyDrive/Colab/DATN/data-modeling/gbm_model_ver3.pkl', 'rb') as file:
    gbm2 = pickle.load(file)

lbl = {}
cols = ['district', 'province', 'ward']
for c in cols:
    with open(os.path.join(save_dir, f'{c}_label_encoder_ver3.pkl'), 'rb') as file:
        lbl[c] = pickle.load(file)

@app.route('/api/predict/gbm', methods=['POST'])
def predict_gbm():
    # Get the input data from the request
    input_data = request.get_json()

    # Create a sample DataFrame from the input_data dictionary
    sample_transformed = pd.DataFrame()
    for c in cols:
        if c in input_data:
            # Check if the input data contains new values that were not in the training data
            new_values = list(set(input_data[c]) - set(lbl[c].classes_))
            if new_values:
                # Add the new values to the LabelEncoder classes and transform the input data
                lbl[c].classes_ = np.concatenate((lbl[c].classes_, new_values))
            sample_transformed[c] = lbl[c].transform(input_data[c])

    cols_num = ['square', 'floor', 'bathroom', 'bedroom']
    for c in cols_num:
        sample_transformed[c] = float(input_data[c])
    column_order = ['square', 'floor', 'bathroom', 'bedroom', 'district', 'province', 'ward']
    sample_transformed = sample_transformed.reindex(columns=column_order)

    # Make the prediction using the loaded model
    y_pred = gbm2.predict(sample_transformed, num_iteration=gbm2.best_iteration_)

    # Return the prediction as a response
    prediction_value = y_pred[0]
    response = {
        "prediction": prediction_value
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run()  # Start the Flask app

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e56a-104-198-135-185.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Jul/2023 11:17:00] "POST /api/predict/gbm HTTP/1.1" 200 -
